In [ ]:
import os
import shutil
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


In [2]:

# -------------------- DATASET PREPARATION --------------------

# Define dataset paths (Replace with your actual path)
DATA_DIR = r"C:\Users\farah\Desktop\ModelsTypesRun\new updates"
TRAIN_DIR = os.path.join(r"C:\Users\farah\Desktop\ModelsTypesRun", "train")
VAL_DIR = os.path.join(r"C:\Users\farah\Desktop\ModelsTypesRun", "val")
TEST_DIR = os.path.join(r"C:\Users\farah\Desktop\ModelsTypesRun", "test")

# Class names (Ensure these match actual folder names inside DATA_DIR)
CLASS_NAMES = ["new mild", "new moderate", "new severe"]  # Change if needed

# Image parameters
IMG_SIZE = (224, 224)  # DenseNet121 default input size
BATCH_SIZE = 32  # Adjust as needed
EPOCHS = 20  # Number of training epochs
LEARNING_RATE = 0.0001  # Optimizer learning rate


def prepare_dataset(data_dir, train_dir, val_dir, test_dir, train_split=0.7, val_split=0.15):
    """ Splits dataset into train, validation, and test sets. """

    # Delete old directories and recreate them
    for dir_path in [train_dir, val_dir, test_dir]:
        if os.path.exists(dir_path):
            shutil.rmtree(dir_path)
        os.makedirs(dir_path, exist_ok=True)

    for class_name in CLASS_NAMES:
        class_path = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_path):
            print(f"Warning: No directory found for class '{class_name}'")
            continue

        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if not images:
            print(f"Warning: No valid images found in '{class_name}'")
            continue

        # Split dataset
        train_images, temp_images = train_test_split(images, train_size=train_split + val_split, stratify=[class_name] * len(images), random_state=42)
        val_images, test_images = train_test_split(temp_images, train_size=val_split / (1 - train_split), stratify=[class_name] * len(temp_images), random_state=42)

        # Copy images to respective directories
        for split, img_list in zip([train_dir, val_dir, test_dir], [train_images, val_images, test_images]):
            split_class_dir = os.path.join(split, class_name)
            os.makedirs(split_class_dir, exist_ok=True)

            for img in img_list:
                src = os.path.join(class_path, img)
                dst = os.path.join(split_class_dir, img)
                shutil.copy(src, dst)

    print("Dataset successfully prepared!\n")


# Run dataset preparation
prepare_dataset(DATA_DIR, TRAIN_DIR, VAL_DIR, TEST_DIR)


Dataset successfully prepared!



In [3]:

# -------------------- IMAGE DATA LOADING --------------------

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load dataset
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

print("\nClass indices:", train_generator.class_indices)


Found 12639 images belonging to 3 classes.
Found 1114 images belonging to 3 classes.
Found 1118 images belonging to 3 classes.

Class indices: {'new mild': 0, 'new moderate': 1, 'new severe': 2}


In [4]:

# -------------------- BUILDING DENSENET121 MODEL --------------------

# Load DenseNet121 as base model (without top layers)
base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers (fine-tuning later)
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
output_layer = Dense(len(CLASS_NAMES), activation="softmax")(x)

# Create final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss="categorical_crossentropy", metrics=["accuracy"])

# -------------------- TRAINING THE MODEL --------------------

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

# Save the model
model.save("densenet121_psoriasis_model.h5")


c:\Users\farah\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 861s 2s/step - accuracy: 0.5447 - loss: 1.0342 - val_accuracy: 0.6903 - val_loss: 0.7393
Epoch 2/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 722s 2s/step - accuracy: 0.6495 - loss: 0.8062 - val_accuracy: 0.6993 - val_loss: 0.7083
Epoch 3/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 682s 2s/step - accuracy: 0.6727 - loss: 0.7647 - val_accuracy: 0.7136 - val_loss: 0.6888
Epoch 4/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 697s 2s/step - accuracy: 0.6867 - loss: 0.7350 - val_accuracy: 0.7199 - val_loss: 0.6698
Epoch 5/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 662s 2s/step - accuracy: 0.6895 - loss: 0.7194 - val_accuracy: 0.7244 - val_loss: 0.6567
Epoch 6/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 700s 2s/step - accuracy: 0.6940 - loss: 0.7185 - val_accuracy: 0.7110 - val_loss: 0.6672
Epoch 7/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 712s 2s/step - accuracy: 0.7010 - loss: 0.7030 - val_accuracy: 0.7154 - val_loss: 0.6796
Epoch 8/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 721s 2s/step - accuracy: 0.7085 - loss: 0.6878 - val_accu

In [5]:

# -------------------- EVALUATION --------------------

# Evaluate on test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")

# Predictions
y_true = test_generator.classes
y_pred = np.argmax(model.predict(test_generator), axis=1)

# Confusion Matrix & Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


35/35 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.8088 - loss: 0.4773

Test Accuracy: 72.27%
35/35 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step

Classification Report:
              precision    recall  f1-score   support

    new mild       0.80      0.82      0.81       592
new moderate       0.54      0.43      0.48       191
  new severe       0.67      0.73      0.69       335

    accuracy                           0.72      1118
   macro avg       0.67      0.66      0.66      1118
weighted avg       0.72      0.72      0.72      1118


Confusion Matrix:
[[483  37  72]
 [ 59  82  50]
 [ 59  33 243]]
